### 方法
```
while 調べるべき特徴量がある限り :
    for 調べる特徴量 in 調べる特徴量の数 :
        ある特徴量ベクトルxiと元のy(y^)の内積を求める(t^i)
        for 100程度 :
            y^を、{t0 = t^0かつt1 = t^1...tj = t^j}という条件の元サンプリング（nealを使う）しyを生成する。
            ある特徴量ベクトルxiとyの内積を求める(ti)
        複数のtiを求めることによって得られた分布に対するt^iの位置から、p値を求める。
    p値の一番小さいXsを知る。
    条件{}にts = t^sを足す（更新する）
    調べる特徴量からXsを除外（更新する）
```

#### 条件について
- t0 = t^0 : 1の総和が同じ
- tj = t^j : xの特徴量jつ目とyの内積の値が同じ

### データ
mushroomの50サンプル分

### yのサンプリング
- コンパイルを行うまでは`pyqubo`
- SAは`SimulatedAnnealingSampler()`

### 改訂(08/03)
- ori_tiのkeyがなかった場合に1とする必要がない
- sum([j[1] for j in ti_list])は必ず100である

### 改訂（08/04）
- samplingでnum=100とする。

In [1]:
#!pip3 install dwave-neal

In [2]:
import pandas as pd 
import numpy as np
from neal import SimulatedAnnealingSampler
from pyqubo import Array, Constraint, Placeholder, solve_qubo
import sys

In [3]:
X_mushroom = pd.read_csv("../input/X_l_mushroom.csv", sep=',', index_col=0)
y_mushroom = pd.read_csv('../input/y_l_mushroom.csv', sep=',', index_col=0)['21']

num_samples = X_mushroom.shape[0]
num_cols = X_mushroom.shape[1]
columns = list(X_mushroom.columns)

y_sum = sum(y_mushroom)

In [4]:
num_samples

50

In [5]:
y_sum

26

In [6]:
def make_y(minP_cols):
    # BINARY変数
    y = Array.create('y', shape=num_samples, vartype='BINARY')
    
    #QUBO式で定式化
    H = (sum(y) - y_sum)**2
    
    if len(minP_cols) > 0:
        for i in minP_cols:
            H_plus = (np.dot(X_mushroom[i], y) - np.dot(X_mushroom[i], y_mushroom))**2
            H = H + H_plus
    
    bqm = H.compile().to_dimod_bqm()

    sa_sampler = SimulatedAnnealingSampler()

    res = sa_sampler.sample(bqm)

    return pd.Series(list(list(res.record)[0][0]))

In [7]:
minP_cols = []
servey_num_col_list = columns
distri_element_num = 100

while len(servey_num_col_list) > 0:    
    p = {}
    for i in servey_num_col_list:
        ori_ti = int(np.dot(X_mushroom[i], y_mushroom)) #特徴量iを表すp
        
        ti_dic = {} #{内積の値:該当数}という辞書。
        
        for _ in range(distri_element_num): 
            y = make_y(minP_cols)
            
            if int(np.dot(X_mushroom[i], y)) in ti_dic.keys():
                ti_dic[int(np.dot(X_mushroom[i], y))] += 1
            else:
                ti_dic[int(np.dot(X_mushroom[i], y))] = 1
                
        count_better_t = sum([v for k, v in ti_dic.items() if k >= ori_ti])
        p[i] = count_better_t / distri_element_num

        
    minP_keys = [k for k, v in p.items() if v == min(list(p.values()))]
    minP_cols += minP_keys
    servey_num_col_list = [i for i in servey_num_col_list if i not in minP_keys]

In [8]:
minP_cols

['1',
 '21',
 '27',
 '38',
 '56',
 '94',
 '100',
 '2',
 '6',
 '14',
 '25',
 '31',
 '37',
 '44',
 '45',
 '49',
 '50',
 '51',
 '60',
 '65',
 '67',
 '74',
 '76',
 '78',
 '83',
 '88',
 '97',
 '99',
 '101',
 '107',
 '108',
 '111',
 '112',
 '115',
 '18',
 '113',
 '52',
 '117',
 '118',
 '0',
 '3',
 '4',
 '5',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '15',
 '16',
 '17',
 '19',
 '20',
 '22',
 '23',
 '24',
 '26',
 '28',
 '29',
 '30',
 '32',
 '33',
 '34',
 '35',
 '36',
 '39',
 '40',
 '41',
 '42',
 '43',
 '46',
 '47',
 '48',
 '53',
 '54',
 '55',
 '57',
 '58',
 '59',
 '61',
 '62',
 '63',
 '64',
 '66',
 '68',
 '69',
 '70',
 '71',
 '72',
 '73',
 '75',
 '77',
 '79',
 '80',
 '81',
 '82',
 '84',
 '85',
 '86',
 '87',
 '89',
 '90',
 '91',
 '92',
 '93',
 '95',
 '96',
 '98',
 '102',
 '103',
 '104',
 '105',
 '106',
 '109',
 '110',
 '114',
 '116']